# Assistente Virtual

Este projeto é um assistente virtual simples desenvolvido para demonstrar como é possível criar um sistema de interação por voz utilizando Processamento de Linguagem Natural (PLN) e reconhecimento de fala. O sistema é baseado em duas funcionalidades principais: Text to Speech (TTS) e Speech to Text (STT), permitindo que o assistente converta textos em áudio e entenda comandos de voz para realizar ações automatizadas.

### Comandos de Voz Disponíveis
O sistema pode executar os seguintes comandos de voz:

--------------------------------------
**"search wikipedia"**

O assistente pedirá ao usuário para dizer o que pesquisar no Wikipedia e retornará um resumo do artigo correspondente.

--------------------------------------
**"open youtube"**

O assistente pedirá ao usuário para dizer o que procurar no YouTube e abrirá uma pesquisa com os resultados relacionados.

--------------------------------------
**"tell me a joke"**

O assistente irá contar uma piada usando a biblioteca pyjokes.

--------------------------------------
**"exit"**

Finaliza a execução do assistente, dizendo "Goodbye".

--------------------------------------

In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
import webbrowser
import wikipedia
import pyjokes
import pygame
import time


def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = f"voice_{int(time.time())}.mp3"
    
    # Excluindo o arquivo se ele já existir
    try:
        os.remove(filename)
    except OSError:
        pass  # Se o arquivo não existir, não faz nada
    
    # Salva o novo arquivo de áudio
    tts.save(filename)

    # Usando o pygame para tocar o áudio
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    # Aguarda até que o áudio termine de ser reproduzido
    while pygame.mixer.music.get_busy():  # Aguarda até que o áudio termine
        pygame.time.Clock().tick(10)

# Módulo de Speech to Text (STT)
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        print("Listening for a command...")
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(f"You said: {said}")
        except sr.UnknownValueError:
            speak("Sorry, I did not understand that.")
        except sr.RequestError:
            speak("Sorry, the service is not available.")
    return said.lower()

# Função para executar comandos
def respond(text):
    print("Executing command based on recognized text: " + text)
    if 'search wikipedia' in text:
        speak("What do you want to search on Wikipedia?")
        query = get_audio()
        if query:
            result = wikipedia.summary(query, sentences=2)
            speak("According to Wikipedia")
            print(result)
            speak(result)
    elif 'open youtube' in text:
        speak("What do you want to search on YouTube?")
        query = get_audio()
        if query:
            url = f"https://www.youtube.com/results?search_query={query}"
            webbrowser.open(url)
            speak(f"Here are the results for {query} on YouTube.")
    elif 'tell me a joke' in text:
        speak(pyjokes.get_joke())
    elif 'exit' in text:
        speak("Goodbye, till next time.")
        exit()

# Sistema de loop para aguardar e responder comandos
while True:
    text = get_audio()
    respond(text)
